In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_groq import ChatGroq
import os

llm = ChatGroq(
    api_key=os.getenv("GROQ_API_KEY"),
    model = 'llama-3.1-70b-versatile',
    streaming = True,
    max_retries=2
)

In [3]:

from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-43863?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
str(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


'Apply for Software Engineer III, ITC\n\nSearch JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologySoftware Engineer III, ITCKarnataka, IndiaBecome a Part of the NIKE, Inc. TeamNIKE, Inc

In [4]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE
    {page_data}
    ### INSTRUCTIONS
    The scraped text is about the job description from a company's job posting.
    Your job is to extract the job posting and return them in JSON format containing the following fields:
    - role
    - experience 
    - skills
    - description
    
    Only return the valid JSON.
    ### VALID JSON(NO PREAMBLE):
    """
)

chain_extract_data = prompt_extract | llm
result = chain_extract_data.invoke(input={"page_data": page_data})
type(result.content)


str

In [5]:
result.content

'```json\n{\n    "role": "Software Engineer III, ITC",\n    "experience": "5-8 years of technology related experience required. 5+ years of hands-on experience in software engineering developing full stack software solutions.",\n    "skills": [\n        "Marketing technology experience",\n        "Experience with various Adobe toolset in the marketing space– in particular Adobe Experience Platform (AEP), Adobe Journey Optimizer (AJO), Customer Journey Analytics(CJA)",\n        "Handson Experience on building marketing use cases using AJO and Segments",\n        "Experience in building customized marketing reporting solutions – Titan,Bigpanda,Splunk",\n        "Experience on Python or Java, Sql, Data Engineering, experience with the following tools or similar: Airflow, EMR, Hive, Spark, or similar",\n        "Demonstrated strength in data modeling, ETL development, and data streaming",\n        "Knowledge of data management fundamentals and data storage principles",\n        "Experience

In [6]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(result.content)
json_res

{'role': 'Software Engineer III, ITC',
 'experience': '5-8 years of technology related experience required. 5+ years of hands-on experience in software engineering developing full stack software solutions.',
 'skills': ['Marketing technology experience',
  'Experience with various Adobe toolset in the marketing space– in particular Adobe Experience Platform (AEP), Adobe Journey Optimizer (AJO), Customer Journey Analytics(CJA)',
  'Handson Experience on building marketing use cases using AJO and Segments',
  'Experience in building customized marketing reporting solutions – Titan,Bigpanda,Splunk',
  'Experience on Python or Java, Sql, Data Engineering, experience with the following tools or similar: Airflow, EMR, Hive, Spark, or similar',
  'Demonstrated strength in data modeling, ETL development, and data streaming',
  'Knowledge of data management fundamentals and data storage principles',
  'Experience in microservices architecture, domain driven design, and RESTful API’s',
  'Hands-

In [7]:
import pandas as pd
df = pd.read_csv('../data/my_portfolio.csv')
df.sample(5)

,Techstack,Links
15,"Backend, Kotlin, Spring Boot",https://example.com/kotlin-backend-portfolio
13,"iOS, Swift, ARKit",https://example.com/ios-ar-portfolio
12,"Android TV, Kotlin, Android NDK",https://example.com/android-tv-portfolio
14,"Cross-platform, Xamarin, Azure",https://example.com/xamarin-portfolio
10,"Android, Java, Room Persistence",https://example.com/android-portfolio


In [8]:
import chromadb
import uuid

client = chromadb.PersistentClient('../vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [9]:
job = json_res
job['skills']

['Marketing technology experience',
 'Experience with various Adobe toolset in the marketing space– in particular Adobe Experience Platform (AEP), Adobe Journey Optimizer (AJO), Customer Journey Analytics(CJA)',
 'Handson Experience on building marketing use cases using AJO and Segments',
 'Experience in building customized marketing reporting solutions – Titan,Bigpanda,Splunk',
 'Experience on Python or Java, Sql, Data Engineering, experience with the following tools or similar: Airflow, EMR, Hive, Spark, or similar',
 'Demonstrated strength in data modeling, ETL development, and data streaming',
 'Knowledge of data management fundamentals and data storage principles',
 'Experience in microservices architecture, domain driven design, and RESTful API’s',
 'Hands-on experience working in a distributed/cloud-based environment with high transaction volumes',
 'Familiarity with the challenge of moving large files across a geographically disperse network',
 'Knowledge of different accelerat

In [10]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/full-stack-js-portfolio'},
  {'links': 'https://example.com/kotlin-android-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/flutter-portfolio'}],
 [{'links': 'https://example.com/kotlin-android-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/vue-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/ios-portfolio'}],
 [{'links': 'https://example.com/full-stack-js-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}

In [11]:
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION
    {job}
    ### INSTRUCTIONS
    Your are a Business Development Manager at JP Morgan. JP Morgan is a AI, Data Science and Data Engineering Consulting company facilitating the seamless integration of AI and Data Science into businesses through the use of cutting-edge technologies. 
    Your Job is to write cold emails to the clients regarding the job description metioned above describing the capabilities of JP Morgan and how it can help the clients in their business.
    Also add the most relevant links from the portfolio to the email to showcase the capabilities of JP Morgan {link_list}.
    Remeber you are Mohan, a Business Development Manager at JP Morgan and write the email accordingly.
    Do not provide preamble.
    ### EMAIL(NO PREAMBLE):
      
    """
)

chain_write_email = prompt_email | llm
result_email = chain_write_email.invoke(input={"job": str(job), "link_list": links})


In [12]:
print(result_email.content)

Subject: Unlock the Power of Customer Data with JP Morgan's Expertise

Dear Hiring Manager,

I came across the Software Engineer III, ITC role at Nike and was impressed by the company's commitment to leveraging technology to drive business growth. As a Business Development Manager at JP Morgan, I'd like to introduce you to our AI, Data Science, and Data Engineering consulting services that can help take your Customer Data Platform to the next level.

Our team has extensive experience in developing full-stack software solutions, data engineering, and implementing CI/CD pipelines. We've worked with various Adobe tools, including Adobe Experience Platform (AEP), Adobe Journey Optimizer (AJO), and Customer Journey Analytics (CJA). Our expertise in data modeling, ETL development, and data streaming can help you optimize your marketing reporting solutions and drive better business outcomes.

I'd like to highlight a few relevant projects from our portfolio that demonstrate our capabilities:

